In [2]:
from google.cloud import translate_v2
import numpy as np
import snorkel
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import MajorityLabelVoter
import json
import pandas as pd

In [3]:

keywords = {
    "alarm": [
        "alarm",
        "wake+up"
    ],
    "audio": [
        "mute",
        "volume",
        " loud",
        "quiet"
    ],
    "iot": [
        "light",
        "wemo",
        "coffee"
    ],
    "calendar": [
        "calendar",
        "schedule",
        "remind"
    ],
    "play": [
        "play ",
        "podcast",
        "audiobook"
    ],
    "general": [
        "good morning",
        "joke",
        "explain"
    ],
    "datetime": [
        "date+today",
        "time+is",
        "date+is"
    ],
    "takeaway": [
        "takeaway",
        "delivery",
        "order"
    ],
    "news": [
        "news",
        "times",
        "headline"
    ],
    "music": [
        "what+song",
        "save+song",
        "shuffle"
    ],
    "weather": [
        "weather",
        "temperature",
        " rain",
        " snow"
    ],
    "qa": [
        "stock",
        "what's",
        "define",
        "describe",
        "what is"
    ],
    "social": [
        "message",
        "tweet",
        "twitter",
        "facebook",
        "complain"
    ],
    "recommendation": [
        "recommend",
        "suggest",
        "restaurant"
    ],
    "cooking": [
        "recipe",
        "timer",
        "cook"
    ],
    "transport": [
        "ticket",
        "train",
        "flight",
        "accident",
        "traffic"
    ],
    "email": [
        "email",
        "inbox"
    ],
    "lists": [
        " list",
        "create+list",
        "delete+list"
    ]
}

In [4]:
def keywords_to_LFs(keyword_dict):
    lfs = []
    for l, v in keyword_dict.items():
        for k in v:
            lfs.append(LabelingFunction(name=f"lf_{k}", f=_keyword_LF, resources={'keyword':k, 'label':l}))
    return lfs

In [6]:
def massive_to_df(dataset):
    indices = [int(i) for i in dataset.keys()]
    text = [ dataset[str(i)]['data']['text'] for i in range(len(indices))]    
    labels = [ dataset[str(i)]['label'] for i in range(len(indices))]    
    data_dict = {'text': text, 'labels': labels}
    df = pd.DataFrame(data=data_dict, index=indices)
    return df
    

In [8]:
# Loading the data 
dataset_name = "massive-en"

idx_to_label = json.load(open(f"../weak_datasets/{dataset_name}/label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
valid_df = massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r")))
train_df = massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r")))
test_df = massive_to_df(json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r")))

# Sample a dev set to help seed ideas for LFs
dev_df = train_df.sample(250, random_state=123)


In [9]:
dev_df

,text,labels
9155,the origins of the universe,5
304,alexa what are amd stock prices,11
1626,what to do next,3
5870,is the largest ocean in the world the pacific,11
7100,give me a list of lists,17
...,...,...
8717,time now in new york,6
2859,lights out,2
3932,date of joining,3
4641,social info,12
